In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras.engine.training import slice_X
from keras.layers.core import Activation, TimeDistributedDense, RepeatVector
from keras.layers import recurrent
import numpy as np
from six.moves import range

Using Theano backend.


In [2]:
class CharacterTable(object):
    '''
    Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilties to their character output
    '''
    def __init__(self, chars, maxlen):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
        self.maxlen = maxlen

    def encode(self, C, maxlen=None):
        maxlen = maxlen if maxlen else self.maxlen
        X = np.zeros((maxlen, len(self.chars)))
        for i, c in enumerate(C):
            X[i, self.char_indices[c]] = 1
        return X

    def decode(self, X, calc_argmax=True):
        if calc_argmax:
            X = X.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in X)


In [3]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [4]:
# Parameters for the model and dataset
TRAINING_SIZE = 50000
DIGITS = 3
INVERT = True

In [5]:
# Try replacing GRU, or SimpleRNN
RNN = recurrent.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1
MAXLEN = DIGITS + 1 + DIGITS #Sentence LEN

In [6]:
chars = '0123456789+ '
ctable = CharacterTable(chars, MAXLEN)

In [7]:
# Let's understand the syntax below
print(sorted(set(chars)))

[' ', '+', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [8]:
questions = []
expected = []
seen = set()

In [9]:
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that X+Y == Y+X (hence the sorting)
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1
    ans += ' ' * (DIGITS + 1 - len(ans))
    if INVERT:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 50000


In [10]:
print('Vectorization...')
# Return a new array of given shape and type, filled with zeros.
X = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    X[i] = ctable.encode(sentence, maxlen=MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, maxlen=DIGITS + 1)

Vectorization...


In [11]:
# just lets take a look at the content of the questions (5 first) and the encoding
for i, sentence in enumerate(questions):
    print(i," = ",sentence)
    print(ctable.encode(sentence, maxlen=MAXLEN))
    if i == 5: break

0  =    0+472
[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]]
1  =  354+008
[[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]]
2  =      0+0
[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0. 

In [12]:
# Shuffle (X, y) in unison as the later parts of X will almost all be larger digits
# np.arrange = Return evenly spaced values within a given interval.
indices = np.arange(len(y))
# Modify a sequence in-place by shuffling its contents.
np.random.shuffle(indices)
X = X[indices]
y = y[indices]

In [13]:
print(np.arange(len(y)))
print(indices)

[    0     1     2 ..., 49997 49998 49999]
[22489  8242 16753 ..., 46566  5941 32472]


In [14]:
# Explicitly set apart 10% for validation data that we never train over
split_at = len(X) - len(X) / 10
(X_train, X_val) = (slice_X(X, 0, split_at), slice_X(X, split_at))
(y_train, y_val) = (y[:split_at], y[split_at:])

print(X_train.shape)
print(y_train.shape)

(45000, 7, 12)
(45000, 4, 12)


/Users/som/Python_DevEnv/hs/lib/python3.4/site-packages/keras/engine/training.py:279: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return X[start:stop]
/Users/som/Python_DevEnv/hs/lib/python3.4/site-packages/ipykernel/__main__.py:4: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [15]:
print('Build model...')
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE
# note: in a situation where your input sequences have a variable length,
# use input_shape=(None, nb_feature).

# http://keras.io/layers/recurrent/#recurrent
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))

# For the decoder's input, we repeat the encoded input for each time step
# http://keras.io/layers/core/#repeatvector
model.add(RepeatVector(DIGITS + 1))

# The decoder RNN could be multiple layers stacked or a single layer
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

# For each of step of the output sequence, decide which character should be chosen
# Apply a same Dense layer for each dimension[1] (time_dimension) input. 
# Especially useful after a recurrent network with 'return_sequence=True'.
# http://keras.io/layers/core/#timedistributeddense
model.add(TimeDistributedDense(len(chars)))

# Applies an activation function to an output.
# http://keras.io/activations/
model.add(Activation('softmax'))

# Configures the model for training
# http://keras.io/models/model/#useful-attributes-of-model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Build model...


/Users/som/Python_DevEnv/hs/lib/python3.4/site-packages/keras/layers/core.py:978: UserWarning: TimeDistributedDense is deprecated, please use TimeDistributed(Dense(...)) instead.
  warnings.warn('TimeDistributedDense is deprecated, '


In [16]:
# Lets explore the model
# http://keras.io/models/about-keras-models/

print(model.summary()) #  summary representation of your model

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
lstm_1 (LSTM)                      (None, 128)         72192       lstm_input_1[0][0]               
____________________________________________________________________________________________________
repeatvector_1 (RepeatVector)      (None, 4, 128)      0           lstm_1[0][0]                     
____________________________________________________________________________________________________
lstm_2 (LSTM)                      (None, 4, 128)      131584      repeatvector_1[0][0]             
____________________________________________________________________________________________________
timedistributeddense_1 (TimeDistrib(None, 4, 12)       1548        lstm_2[0][0]                     
___________________________________________________________________________________________

In [17]:
print(model.get_config()) # dictionary containing the configuration of the model

[{'class_name': 'LSTM', 'config': {'stateful': False, 'unroll': False, 'init': 'glorot_uniform', 'name': 'lstm_1', 'input_length': None, 'dropout_U': 0.0, 'input_dim': 12, 'output_dim': 128, 'activation': 'tanh', 'U_regularizer': None, 'b_regularizer': None, 'input_dtype': 'float32', 'inner_activation': 'hard_sigmoid', 'return_sequences': False, 'consume_less': 'cpu', 'go_backwards': False, 'batch_input_shape': (None, 7, 12), 'trainable': True, 'forget_bias_init': 'one', 'dropout_W': 0.0, 'inner_init': 'orthogonal', 'W_regularizer': None}}, {'class_name': 'RepeatVector', 'config': {'name': 'repeatvector_1', 'trainable': True, 'n': 4}}, {'class_name': 'LSTM', 'config': {'stateful': False, 'U_regularizer': None, 'b_regularizer': None, 'unroll': False, 'activation': 'tanh', 'init': 'glorot_uniform', 'name': 'lstm_2', 'return_sequences': True, 'inner_activation': 'hard_sigmoid', 'consume_less': 'cpu', 'go_backwards': False, 'input_length': None, 'inner_init': 'orthogonal', 'trainable': Tru

In [18]:
print(model.get_weights()) # list of all weight tensors in the model, as Numpy arrays

[array([[-0.05688991, -0.20616724, -0.03885408, ..., -0.05425313,
        -0.08791005,  0.0221867 ],
       [ 0.10702851, -0.11690623,  0.19620807, ..., -0.0371106 ,
         0.09570221,  0.11045586],
       [ 0.19467279, -0.08715624, -0.0315129 , ...,  0.09902659,
        -0.14890358,  0.08740961],
       ..., 
       [ 0.14905863,  0.01145949,  0.0081084 , ..., -0.09142116,
        -0.06175877, -0.0067453 ],
       [-0.0511357 , -0.1089561 , -0.04776838, ...,  0.16634578,
        -0.0491608 , -0.20180464],
       [ 0.00033753, -0.07128713,  0.18035302, ...,  0.04103767,
        -0.06444315, -0.05630661]], dtype=float32), array([[ 0.01936707, -0.01729723,  0.03921725, ...,  0.10109361,
         0.01403662,  0.04996431],
       [-0.11023977,  0.03153499,  0.12532127, ..., -0.03253487,
        -0.04452768, -0.01921306],
       [-0.15613616,  0.00373495,  0.02013076, ..., -0.02240739,
        -0.0264896 , -0.20991005],
       ..., 
       [ 0.1620263 , -0.06731363,  0.01463934, ...,  0.0

In [19]:
print(model.to_json()) # model as a JSON string

{"class_name": "Sequential", "config": [{"class_name": "LSTM", "config": {"stateful": false, "unroll": false, "init": "glorot_uniform", "name": "lstm_1", "input_length": null, "dropout_U": 0.0, "input_dim": 12, "output_dim": 128, "activation": "tanh", "U_regularizer": null, "b_regularizer": null, "input_dtype": "float32", "inner_activation": "hard_sigmoid", "return_sequences": false, "consume_less": "cpu", "go_backwards": false, "batch_input_shape": [null, 7, 12], "trainable": true, "forget_bias_init": "one", "dropout_W": 0.0, "inner_init": "orthogonal", "W_regularizer": null}}, {"class_name": "RepeatVector", "config": {"name": "repeatvector_1", "trainable": true, "n": 4}}, {"class_name": "LSTM", "config": {"stateful": false, "U_regularizer": null, "b_regularizer": null, "unroll": false, "activation": "tanh", "init": "glorot_uniform", "name": "lstm_2", "return_sequences": true, "inner_activation": "hard_sigmoid", "consume_less": "cpu", "go_backwards": false, "input_length": null, "inne

In [ ]:
# Train the model each generation and show predictions against the validation dataset
# Number of iterations = number of passes, each pass using [batch size] number of examples. 
# To be clear, one pass = one forward pass + one backward pass 
# (we do not count the forward pass and backward pass as two different passes).
for iteration in range(1, 200):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X_train, y_train, batch_size=BATCH_SIZE, nb_epoch=1,
              validation_data=(X_val, y_val))
    ###
    # Select 10 samples from the validation set at random so we can visualize errors
    for i in range(10):
        ind = np.random.randint(0, len(X_val))
        rowX, rowy = X_val[np.array([ind])], y_val[np.array([ind])]
        # Prediction of the model baesd on the rowX
        preds = model.predict_classes(rowX, verbose=0)
        print(preds[0])
        # Decoding
        q = ctable.decode(rowX[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        # Printing
        print('Q', q[::-1] if INVERT else q)
        print('T', correct)
        print(colors.ok + '☑' + colors.close if correct == guess else colors.fail + '☒' + colors.close, guess)
        print('---')
        
        ()


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 22s - loss: 1.8471 - acc: 0.3319 - val_loss: 1.7171 - val_acc: 0.3657
[3 3 0 0]
Q 26+3   
T 29  
☒ 11  
---

[5 5 5 0]
Q 23+450 
T 473 
☒ 333 
---

[5 5 5 0]
Q 723+20 
T 743 
☒ 333 
---

[3 2 8 0]
Q 195+16 
T 211 
☒ 106 
---

[3 2 3 0]
Q 91+59  
T 150 
☒ 101 
---

[3 2 2 0]
Q 89+202 
T 291 
☒ 100 
---

[4 4 3 0]
Q 12+105 
T 117 
☒ 221 
---

[6 6 5 0]
Q 843+4  
T 847 
☒ 443 
---

[3 2 5 0]
Q 19+440 
T 459 
☒ 103 
---

[3 2 2 0]
Q 114+136
T 250 
☒ 100 
---


--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 22s - loss: 1.6299 - acc: 0.3946 - val_loss: 1.5220 - val_acc: 0.4308
[10  2  8  0]
Q 853+72 
T 925 
☒ 806 
---

[9 5 7 0]
Q 573+1  
T 574 
☒ 735 
---

[11 11  9  0]
Q 51+881 
T 932 
☒ 997 
---

[9 9 